# CMake实践

## 什么是CMake

　　CMake允许开发者编译写一种与平台无关的CMakeList.txt文件来制定整个编译流程，然后根据目标用户的平台进一步生成所需的本地化Makefile和工程文件，如Unix的Makefile或Windows的Visual Studio工程，从而做到"Write once, run everywhere"．

## CMake的主要特点

- 开源代码
- 跨平台
- 能够管理大型项目
- 简化编译构建的编译过程
- 高效
- 可扩展

### 最简例子Helloworld--文件夹t1

基本语法规则：<br>
- 变量使用${}方式取值，但是在IF控制语句中直接使用变量名
- 指令（参数1　参数2...），参数之间使用空格或分号分开<br>
　eg．ADD_EXECUTABLE(hello main.c func.c)或<br>
 　　　ADD_EXECUTABLE(hello main.c;unc.c)
- 指令与大小写无关的，参数和变量与大小写有关，推荐全部使用大写指令
- make clean对构建结果进行清理

文件夹t1进行的是内部构建(in-source build)，而cmake强烈推荐的是外部构建(out-of-source build)．采用out-of-source外部编译的一个最大的好处就是对原有的工程没有任何影响．

**PROJECT** :<br>
PROJECT(projectname [CXX] [C] [Java])<br>
这个指令定义工程名称,并可指定工程支持的语言，支持语言列表可以忽略．<br>
这条指令隐式的定义了两个cmake变量：(projectname)\_BINARY\_DIR以及(projectname)\_SOURCE\_DIR

**SET** :<br>
SET(VAR [VALUE] [CACHE TYPE DOCSTRING [FORCE]])<br>
可以用来显式的定义变量即可

**MESSAGE** :<br>
MESSAGE([SEND_ERROR | STATUS | FATAL_ERROR] "message to display"...)<br>
这指令用于向终端输出用户定义的信息，包括了三种类型：<br>
SEND_ERROR,产生错误,生成过程被跳过<br>
SATUS ,输出前缀为 — 的信息<br>
FATAL_ERROR,立即终止所有 cmake 过程

**ADD_EXECUTABLE**:<br>
ADD_EXECUTABLE(hello ${SRC_LIST}) <br>
定义了这个工程会生成一个文件名为 hello 的可执行文件,相关的源文件是 SRC_LIST 中定义的源文件列表

### 初步优化Helloworld--文件夹t2

采用out-of-source外部构建

- 子目录src，存放工程源代码
- 子目录doc，存放工程文档
- 工程目录存放文本文件COPYRIGHT,README
- 子目录build，存放编译所需的Ｍakefile以及其他的中间文件

**ADD_SUBDIRECTORY**<br>
ADD_SUBDIRECTORY(source_dir [binary_dir] [EXCLUDE_FROM_ALL])<br>
这个指令用于向当前工程添加存放源文件的子目录，并可以指定中间二进制和目标二进制存放的位置，EXCLUDE_FROM_ALL参数的含义是将这个目录从编译过程中排除<br>

**可指定最终的目标二进制和共享库的位置**<br>
**EXECUTABLE_OUTPUT_PATH**和**LIBRARY_OUTPUT_PATH**变量<br>
若是外部编译，指的是外部编译所在目录，在t2就是build目录．<br>
若需要改变目标存放路径，在哪里有ADD_EXECUTABLE或ADD_LIBRARY，就在哪里加入上述的定义．

**如何安装**<br>
- 从代码编译后直接make instal安装
- 打包时指定目录安装<br>
此处引入一个新的cmake指令INSTALL和一个非常有用的变量CMAKE_INSTALL_PREFIX<br>
常见的使用方法：<br>
cmake -DCMAKE_INSTALL_PREFIX=/usr .

**INSTALL**包含各种安装<br>
　1. 目标文件<br>
　2. 普通文件<br>
　3. 非目标文件的可执行程序<br>
　4. 目录<br>

1. 目标文件的安装:<br>
　　INSTALL(TARGETS targets\...<br>
　　[[ARCHIVE|LIBRARY|RUNTIME]<br>
　　[DESTINATION < dir\>]
　　[PERMISSIONS permissions...]<br>
　　[CONFIGURATIONS[Debug|Release|...]]<br>
　　[COMPONENT < component\>]<br>
　　[OPTIONAL]] [...])<br>
　　参数中的 TARGETS 后面跟的就是我们通过 ADD_EXECUTABLE 或者 ADD_LIBRARY 定义的目标文件,可能是可执行二进制、动态库、静态库<br>
　　ARCHIVE 特指静态库, LIBRARY特指动态库, RUNTIME特指可执行目标二进制  

2. 普通文件的安装:<br>
　　INSTALL(FILES files... DESTINATION <dir\><br>
　　[PERMISSIONS permissions...]<br>
　　[CONFIGURATIONS [Debug|Release|...]]<br>
　　[COMPONENT <component\>]<br>
　　[RENAME <name\>] [OPTIONAL])<br>
　　可用于安装一般文件,并可以指定访问权限,文件名是此指令所在路径下的相对路径。如果默认不定义权限PERMISSIONS,安装后的权限为:OWNER_WRITE, OWNER_READ, GROUP_READ,和 WORLD_READ,即 644 权限。

3. 非目标文件的可执行程序安装(比如脚本之类):<br>
　　INSTALL(PROGRAMS files... DESTINATION <dir\><br>
　　[PERMISSIONS permissions...]<br>
　　[CONFIGURATIONS [Debug|Release|...]]<br>
　　[COMPONENT <component\>]<br>
　　[RENAME <name\>] [OPTIONAL])<br>
　　跟上面的 FILES 指令使用方法一样,唯一的不同是安装后权限为: OWNER_EXECUTE, GROUP_EXECUTE, 和 WORLD_EXECUTE,即 755 权限  

4. 目录的安装:<br>
　　INSTALL(DIRECTORY dirs... DESTINATION <dir\><br>
　　[FILE_PERMISSIONS permissions...]<br>
　　[DIRECTORY_PERMISSIONS permissions...]<br>
　　[USE_SOURCE_PERMISSIONS]<br>
　　[CONFIGURATIONS [Debug|Release|...]]<br>
　　[COMPONENT <component\>]<br>
　　[[PATTERN <pattern\> | REGEX <regex\>]<br>
　　[EXCLUDE] [PERMISSIONS permissions...]] [...])<br>
　　DIRECTORY 后面连接的是所在 Source 目录的相对路径,但务必注意:abc 和 abc/有很大的区别。
如果目录名不以/结尾,那么这个目录将被安装为目标路径下的 abc,如果目录名以/结尾,代表将这个目录中的内容安装到目标路径,但不包括这个目录本身。<br>
　　PATTERN 用于使用正则表达式进行过滤,PERMISSIONS 用于指定 PATTERN过滤后的文件
权限。

### 静态库与动态库构建Helloworld--文件夹t3

建立一个静态库和动态库，提供HelloFunc函数供其他程序编程使用<br>
安装头文件与共享库

**ADD_LIBRARY**<br>
　　ADD_LIBRARY(libname [SHARED|STATIC|MODULE]<br>
　　[EXCLUDE_FROM_ALL]　source1 source2 ... sourceN)<br>
　　SHARED: 动态库<br>
　　STATIC: 静态库<br>
　　MODULE: 在使用 dyld 的系统有效,如果不支持 dyld,则被当作 SHARED 对待<br>
不需要写全libxxx.so，只需填写xxx即可，cmake系统会自动生成libxxx.so

ADD_LIBRARY可以添加静态库和动态库，但是此操作不能构建名字相同的静态库和动态库，所以需要以下指令：<br>
**SET_TARGET_PROPERTIES**<br>
SET_TARGET_PROPERTIES(target1 target2 ...
PROPERTIES prop1 value1 prop2 value2 ...)<br>
　　可以用来设置输出的名称,对于动态库,还可以用来指定动态库版本和 API 版本

eg. <br>
　　SET_TARGET_PROPERTIES(hello_static PROPERTIES OUTPUT_NAME "hello")

### 专业版Helloworld--文件夹t4

使用构建的共享库

引入头文件搜索路径<br>
**INCLUDE_DIRECTORIES**<br>
INCLUDE_DIRECTORIES([AFTER|BEFORE] [SYSTEM] dir1 dir2 ...)<br>
用来向工程添加多个特定的头文件搜索路径,路径之间用空格分割,如果路径中包含了空格,可以使用双引号将它括起来,默认的行为是追加到当前的头文件搜索路径的后面,你可以通过两种方式来进行控制搜索路径添加的方式:<br>
　　1,CMAKE_INCLUDE_DIRECTORIES_BEFORE,通过 SET 这个 cmake 变量为 on,可以将添加的头文件搜索路径放在已有路径的前面。<br>
　　2,通过 AFTER 或者 BEFORE 参数,也可以控制是追加还是置前。<br>

为target添加共享库<br>
**LINK_DIRECTORIES**<br>
LINK_DIRECTORIES(directory1 directory2 ...)<br>
添加非标准的共享库搜索路径,比如,在工程内部同时存在共享库和可执行二进制,在编译时就需要指定一下这些共享库的路径。

**TARGET_LINK_LIBRARIES**<br>
TARGET_LINK_LIBRARIES(target library1 <debug | optimized> library2...)<br>
用来为 target 添加需要链接的共享库

eg.<br>
　　TARGET_LINK_LIBRARIES(main hello)<br>
　　TARGET_LINK_LIBRARIES(main libhello.so)

可通过下列方式了解main的情况：<br>
ldd main